In [12]:
!pip install wikipedia cohere pydantic

In [13]:
from typing import List
from pydantic import BaseModel
import wikipedia
import cohere
import json

In [14]:
co = cohere.Client("3JE437qflNp5M7TbvoQXMmeDuRrRWgylGCq09x29")

In [15]:
class InstitutionDetails(BaseModel):
    institution_name: str
    founded_year: int
    founded_by: str
    no_of_employees: str
    branches: List[str]
    courses: List[str]

In [16]:
def fetch_wikipedia_summary(name: str) -> str:
    return wikipedia.summary(name, sentences=4)

In [17]:
details_format = {
    "institution_name": "str",
    "founded_year" :"int",
    "founded_by": "str",
    "no_of_employees": "str",
    "branches": "List[str]",
    "courses": "List[str]",
}

In [18]:
def format_wiki_summary(summary):
    response = co.generate(prompt=f"""
    Summarize the following text: ||{summary}|| in this json format {details_format}
    """)
    text = response.generations[0].text.strip()

    start = text.find("{")
    end = text.rfind("}") + 1
    json_str = text[start:end]

    data = json.loads(json_str)
    return InstitutionDetails(**data)

In [19]:
def get_institution_details() -> InstitutionDetails:
    name = input("Enter institution name: ")
    summary = fetch_wikipedia_summary(name)
    return format_wiki_summary(summary)

In [22]:
institution = get_institution_details()
print("Institution Name: ", institution.institution_name)
print("Founded in: ", institution.founded_year)
print("Founded by: ", institution.founded_by)
print("Employee count: ", institution.no_of_employees)
print("Departments: ", institution.branches)
print("Courses Offered: ", institution.courses)

Enter institution name:  RNS Institute of Technology


Institution Name:  Rama Nagappa Shetty Institute of Technology
Founded in:  2001
Founded by:  Dr.
Employee count:  Over 1000
Departments:  ['Civil', 'Electrical', 'Mechanical', 'Computer Science', 'Chemical', 'Electronics and Communication', 'Bio Technological']
Courses Offered:  ['Bachelor of Engineering', 'Bachelor of Science']
